## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from io import StringIO

pd.set_option('display.max_columns', None)

## Personal Access Token (required to access csv files on private repo)

In [2]:
username = 'hgetac1'
token = '6f13a588d20878f6a4d3a541f16b463ddbb5b31b'

# creates re-usable session object to access url links for raw csv files
github_session = requests.Session()
github_session.auth = (username, token)

## Import Aggregated Dataset

In [3]:
url = "https://raw.githubusercontent.com/Baltimore-Data-Science-Corps-TU-Team/Logistic-Regression-911-Calls/main/wrangled_data/call_data.csv?token=AJZB73I3XL4YRIKL7JKIW5C7UMIQW"

# retrieving raw version of csv file on GitHub
path = github_session.get(url).content


# reading downloaded content and making it into a pandas dataframe
call_data = pd.read_csv(StringIO(path.decode('utf-8')), low_memory=False)

In [4]:
# check if aggregated dataset merged correctly: Do all 11 types of crime exist in the dataset? 
# AGGRAV ASSAULT, ROBBERY ARMED, ARSON, ASSAULT, BURGLARY, COMMON ASSAULT, GAMBLING, LARCENY, NARCOTICS, PROSTITUTION, SHOOTING


found = call_data[call_data['Description'].str.contains('SHOOTING')]
print(found.count())

RecordID                       14449
CallNumber                     14449
CallDate                       14449
CallTime                       14449
DayOfWeek                      14449
Covid-19                       14449
Priority                       14449
Severity                       14449
District                       14449
Description                    14449
IncidentLocation               14449
ZipCode                        13783
Neighborhood                   14432
PoliceDistrict                 14432
PolicePost                     14432
CouncilDistrict                14432
SheriffDistricts               14432
Community_Statistical_Areas    14417
Census_Tracts                  14431
VRIZones                        3368
Location                       14163
dtype: int64


## Viewing Dataset

In [5]:
call_data.head(10)

,RecordID,CallNumber,CallDate,CallTime,DayOfWeek,Covid-19,Priority,Severity,District,Description,IncidentLocation,ZipCode,Neighborhood,PoliceDistrict,PolicePost,CouncilDistrict,SheriffDistricts,Community_Statistical_Areas,Census_Tracts,VRIZones,Location
0,3953549,P151262297,5/6/15,6:22:00 PM,Wednesday,Precovid,Medium,Nonviolent,NW,ARSON,2800 DENISON ST,21216.0,Hanlon-Longwood,Northwestern,641.0,7.0,D9,Greater Mondawmin,Census Tract 1507.01,NaN,"2800 DENISON ST\rBALTIMORE, MD\r(39.319911, -7..."
1,3845555,P151630190,6/12/15,1:36:00 AM,Friday,Precovid,Low,Nonviolent,WD,ARSON,W NORTH AV/RETREAT ST,21217.0,Penn North,Western,733.0,7.0,D9,Greater Mondawmin,Census Tract 1501,Western,"W NORTH AV\rBALTIMORE, MD"
2,3788155,P151810834,6/30/15,9:13:00 AM,Tuesday,Precovid,High,Nonviolent,WD,ARSON,1900 HARLEM AV,21217.0,Midtown-Edmondson,Western,722.0,9.0,D8,Sandtown-Winchester/Harlem Park,Census Tract 1604,NaN,"1900 HARLEM AV\rBALTIMORE, MD\r(39.296311, -76..."
3,3659533,P152212663,8/9/15,8:20:00 PM,Sunday,Precovid,Low,Nonviolent,SE,ARSON,300 BLK N ELLWOOD AV,21224.0,Ellwood Park/Monument,Southeastern,224.0,13.0,D5,Patterson Park North & East,Census Tract 601,NaN,"300 BLK N ELLWOOD AV\rBALTIMORE, MD\r(39.29555..."
4,3289892,P153443369,12/10/15,10:36:00 PM,Thursday,Precovid,High,Nonviolent,SE,ARSON,E FAYETTE ST/N BROADWAY,21231.0,Washington Hill,Southeastern,212.0,12.0,D7,Oldtown/Middle East,Census Tract 301,NaN,"E N FAYETTE ST\rBALTIMORE, MD"
5,3056586,P160660326,3/6/16,2:47:00 AM,Sunday,Precovid,High,Nonviolent,NW,ARSON,3100 SUMTER AV,21215.0,Central Park Heights,Northwestern,614.0,6.0,D1,Southern Park Heights,Census Tract 2716,NaN,"3100 SUMTER AV\rBALTIMORE, MD\r(39.34225, -76...."
6,3054171,P160670263,3/7/16,2:50:00 AM,Monday,Precovid,High,Nonviolent,NW,ARSON,W NORTHERN PY/WABASH AV,21215.0,Reisterstown Station,Northwestern,631.0,5.0,D1,Howard Park/West Arlington,Census Tract 2801.02,NaN,"W NORTHERN PY\rBALTIMORE, MD"
7,2745442,P161683586,6/16/16,11:51:00 PM,Thursday,Precovid,High,Nonviolent,ND,ARSON,1000 CEDARCROFT RD,21212.0,Lake Walker,Northern,522.0,4.0,D2,Chinquapin Park/Belvedere,Census Tract 2708.04,NaN,"1000 CEDARCROFT RD\rBALTIMORE, MD\r(39.369722,..."
8,2669011,P161940215,7/12/16,1:41:00 AM,Tuesday,Precovid,High,Nonviolent,NE,ARSON,RAVENWOOD AV/CHESTERFIELD AV,21213.0,Belair-Edison,Northeastern,432.0,13.0,D6,Belair-Edison,Census Tract 2603.02,NaN,"RAVENWOOD AV\rBALTIMORE, MD"
9,2527869,P162403417,8/27/16,11:34:00 PM,Saturday,Precovid,High,Nonviolent,NE,ARSON,1000 ARMISTEAD WY,21205.0,Armistead Gardens,Northeastern,433.0,2.0,D6,Claremont/Armistead,Census Tract 2604.01,NaN,"1000 ARMISTEAD WY\rBALTIMORE, MD\r(39.303864, ..."


***
##  Regex to manipulate Location Attrribute 

In [6]:
# (1) extract data from location attribute, (2) create columns for lat/lon, (3) assign to new dataframe

df = call_data.Location.str.extract('^(?P<Test_Location>.*)\s*\((?P<Latitude>[^,]*),\s*(?P<Longitude>\S*)\).*$',expand=True)

In [7]:
df

,Test_Location,Latitude,Longitude
0,"2800 DENISON ST\rBALTIMORE, MD\r",39.319911,-76.67474
1,NaN,NaN,NaN
2,"1900 HARLEM AV\rBALTIMORE, MD\r",39.296311,-76.647506
3,"300 BLK N ELLWOOD AV\rBALTIMORE, MD\r",39.295552,-76.573545
4,NaN,NaN,NaN
...,...,...,...
430250,"2400 W FRANKLIN ST\rBALTIMORE, MD\r",39.293515,-76.65486
430251,"3600 CLARKS LN\rBALTIMORE, MD\r",39.361939,-76.70028
430252,"5900 RADECKE AV\rBALTIMORE, MD\r",39.330693,-76.532031
430253,"5200 YORK RD\rBALTIMORE, MD\r",39.352315,-76.609786


In [8]:
# drop NaN rows

df.dropna(axis=0)

,Test_Location,Latitude,Longitude
0,"2800 DENISON ST\rBALTIMORE, MD\r",39.319911,-76.67474
2,"1900 HARLEM AV\rBALTIMORE, MD\r",39.296311,-76.647506
3,"300 BLK N ELLWOOD AV\rBALTIMORE, MD\r",39.295552,-76.573545
5,"3100 SUMTER AV\rBALTIMORE, MD\r",39.34225,-76.667125
7,"1000 CEDARCROFT RD\rBALTIMORE, MD\r",39.369722,-76.601216
...,...,...,...
430250,"2400 W FRANKLIN ST\rBALTIMORE, MD\r",39.293515,-76.65486
430251,"3600 CLARKS LN\rBALTIMORE, MD\r",39.361939,-76.70028
430252,"5900 RADECKE AV\rBALTIMORE, MD\r",39.330693,-76.532031
430253,"5200 YORK RD\rBALTIMORE, MD\r",39.352315,-76.609786


***
## Concatenate orginal dataframe w/ extracted dataframe

In [9]:
concatenated = pd.concat([call_data, df], axis=1)

In [10]:
concatenated

,RecordID,CallNumber,CallDate,CallTime,DayOfWeek,Covid-19,Priority,Severity,District,Description,IncidentLocation,ZipCode,Neighborhood,PoliceDistrict,PolicePost,CouncilDistrict,SheriffDistricts,Community_Statistical_Areas,Census_Tracts,VRIZones,Location,Test_Location,Latitude,Longitude
0,3953549,P151262297,5/6/15,6:22:00 PM,Wednesday,Precovid,Medium,Nonviolent,NW,ARSON,2800 DENISON ST,21216.0,Hanlon-Longwood,Northwestern,641.0,7.0,D9,Greater Mondawmin,Census Tract 1507.01,NaN,"2800 DENISON ST\rBALTIMORE, MD\r(39.319911, -7...","2800 DENISON ST\rBALTIMORE, MD\r",39.319911,-76.67474
1,3845555,P151630190,6/12/15,1:36:00 AM,Friday,Precovid,Low,Nonviolent,WD,ARSON,W NORTH AV/RETREAT ST,21217.0,Penn North,Western,733.0,7.0,D9,Greater Mondawmin,Census Tract 1501,Western,"W NORTH AV\rBALTIMORE, MD",NaN,NaN,NaN
2,3788155,P151810834,6/30/15,9:13:00 AM,Tuesday,Precovid,High,Nonviolent,WD,ARSON,1900 HARLEM AV,21217.0,Midtown-Edmondson,Western,722.0,9.0,D8,Sandtown-Winchester/Harlem Park,Census Tract 1604,NaN,"1900 HARLEM AV\rBALTIMORE, MD\r(39.296311, -76...","1900 HARLEM AV\rBALTIMORE, MD\r",39.296311,-76.647506
3,3659533,P152212663,8/9/15,8:20:00 PM,Sunday,Precovid,Low,Nonviolent,SE,ARSON,300 BLK N ELLWOOD AV,21224.0,Ellwood Park/Monument,Southeastern,224.0,13.0,D5,Patterson Park North & East,Census Tract 601,NaN,"300 BLK N ELLWOOD AV\rBALTIMORE, MD\r(39.29555...","300 BLK N ELLWOOD AV\rBALTIMORE, MD\r",39.295552,-76.573545
4,3289892,P153443369,12/10/15,10:36:00 PM,Thursday,Precovid,High,Nonviolent,SE,ARSON,E FAYETTE ST/N BROADWAY,21231.0,Washington Hill,Southeastern,212.0,12.0,D7,Oldtown/Middle East,Census Tract 301,NaN,"E N FAYETTE ST\rBALTIMORE, MD",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430250,8386106,P202962331,10/22/20,1:05:00 PM,Thursday,Postcovid,Low,Nonviolent,WD,LARCENY,2400 W FRANKLIN ST,21223.0,Penrose/Fayette Street Outreach,Southwestern,842.0,9.0,D8,Southwest Baltimore,Census Tract 2002,NaN,"2400 W FRANKLIN ST\rBALTIMORE, MD\r(39.293515,...","2400 W FRANKLIN ST\rBALTIMORE, MD\r",39.293515,-76.65486
430251,8386157,P202962370,10/22/20,1:18:00 PM,Thursday,Postcovid,Low,Nonviolent,NW,LARCENY,3600 CLARKS LN,21215.0,Glen,Northwestern,632.0,5.0,D1,Glen-Fallstaff,Census Tract 2720.06,NaN,"3600 CLARKS LN\rBALTIMORE, MD\r(39.361939, -76...","3600 CLARKS LN\rBALTIMORE, MD\r",39.361939,-76.70028
430252,8386240,P202962468,10/22/20,1:50:00 PM,Thursday,Postcovid,Low,Nonviolent,NE,LARCENY,5900 RADECKE AV,21206.0,Cedonia,Northeastern,441.0,2.0,D6,Claremont/Armistead,Census Tract 2604.03,NaN,"5900 RADECKE AV\rBALTIMORE, MD\r(39.330693, -7...","5900 RADECKE AV\rBALTIMORE, MD\r",39.330693,-76.532031
430253,8386262,P202962543,10/22/20,2:19:00 PM,Thursday,Postcovid,Low,Nonviolent,ND,LARCENY,5200 YORK RD,21212.0,Radnor-Winston,Northern,521.0,4.0,D2,North Baltimore/Guilford/Homeland,Census Tract 2711.01,NaN,"5200 YORK RD\rBALTIMORE, MD\r(39.352315, -76.6...","5200 YORK RD\rBALTIMORE, MD\r",39.352315,-76.609786


In [11]:
# drop Test_Location column (from concatenated dataset) and assign it back to call_data

call_data = concatenated.drop(["Test_Location"],axis=1)

In [12]:
call_data

,RecordID,CallNumber,CallDate,CallTime,DayOfWeek,Covid-19,Priority,Severity,District,Description,IncidentLocation,ZipCode,Neighborhood,PoliceDistrict,PolicePost,CouncilDistrict,SheriffDistricts,Community_Statistical_Areas,Census_Tracts,VRIZones,Location,Latitude,Longitude
0,3953549,P151262297,5/6/15,6:22:00 PM,Wednesday,Precovid,Medium,Nonviolent,NW,ARSON,2800 DENISON ST,21216.0,Hanlon-Longwood,Northwestern,641.0,7.0,D9,Greater Mondawmin,Census Tract 1507.01,NaN,"2800 DENISON ST\rBALTIMORE, MD\r(39.319911, -7...",39.319911,-76.67474
1,3845555,P151630190,6/12/15,1:36:00 AM,Friday,Precovid,Low,Nonviolent,WD,ARSON,W NORTH AV/RETREAT ST,21217.0,Penn North,Western,733.0,7.0,D9,Greater Mondawmin,Census Tract 1501,Western,"W NORTH AV\rBALTIMORE, MD",NaN,NaN
2,3788155,P151810834,6/30/15,9:13:00 AM,Tuesday,Precovid,High,Nonviolent,WD,ARSON,1900 HARLEM AV,21217.0,Midtown-Edmondson,Western,722.0,9.0,D8,Sandtown-Winchester/Harlem Park,Census Tract 1604,NaN,"1900 HARLEM AV\rBALTIMORE, MD\r(39.296311, -76...",39.296311,-76.647506
3,3659533,P152212663,8/9/15,8:20:00 PM,Sunday,Precovid,Low,Nonviolent,SE,ARSON,300 BLK N ELLWOOD AV,21224.0,Ellwood Park/Monument,Southeastern,224.0,13.0,D5,Patterson Park North & East,Census Tract 601,NaN,"300 BLK N ELLWOOD AV\rBALTIMORE, MD\r(39.29555...",39.295552,-76.573545
4,3289892,P153443369,12/10/15,10:36:00 PM,Thursday,Precovid,High,Nonviolent,SE,ARSON,E FAYETTE ST/N BROADWAY,21231.0,Washington Hill,Southeastern,212.0,12.0,D7,Oldtown/Middle East,Census Tract 301,NaN,"E N FAYETTE ST\rBALTIMORE, MD",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430250,8386106,P202962331,10/22/20,1:05:00 PM,Thursday,Postcovid,Low,Nonviolent,WD,LARCENY,2400 W FRANKLIN ST,21223.0,Penrose/Fayette Street Outreach,Southwestern,842.0,9.0,D8,Southwest Baltimore,Census Tract 2002,NaN,"2400 W FRANKLIN ST\rBALTIMORE, MD\r(39.293515,...",39.293515,-76.65486
430251,8386157,P202962370,10/22/20,1:18:00 PM,Thursday,Postcovid,Low,Nonviolent,NW,LARCENY,3600 CLARKS LN,21215.0,Glen,Northwestern,632.0,5.0,D1,Glen-Fallstaff,Census Tract 2720.06,NaN,"3600 CLARKS LN\rBALTIMORE, MD\r(39.361939, -76...",39.361939,-76.70028
430252,8386240,P202962468,10/22/20,1:50:00 PM,Thursday,Postcovid,Low,Nonviolent,NE,LARCENY,5900 RADECKE AV,21206.0,Cedonia,Northeastern,441.0,2.0,D6,Claremont/Armistead,Census Tract 2604.03,NaN,"5900 RADECKE AV\rBALTIMORE, MD\r(39.330693, -7...",39.330693,-76.532031
430253,8386262,P202962543,10/22/20,2:19:00 PM,Thursday,Postcovid,Low,Nonviolent,ND,LARCENY,5200 YORK RD,21212.0,Radnor-Winston,Northern,521.0,4.0,D2,North Baltimore/Guilford/Homeland,Census Tract 2711.01,NaN,"5200 YORK RD\rBALTIMORE, MD\r(39.352315, -76.6...",39.352315,-76.609786


***
## Convert call_data to new csv

In [ ]:
call_data.to_csv("911_data.csv")